# Analisis de comentarios de películas

## 1. Importacion de librerias

In [39]:
import numpy as np
import pandas as pd
import re

import matplotlib.pyplot as plt
import seaborn as sns

# Librerias para preprocesamiento de texto
import nltk as nltk
from nltk.corpus import stopwords as stopwords
from nltk.tokenize import word_tokenize as word_tokenize
nltk.download('punkt')

import spacy
nlp = spacy.load('es_core_web_sm')

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import scipy.stats as stats

## 2. Carga de datos

In [40]:
# Carga del csv
file = './data/MovieReviews.csv'
raw = pd.read_csv(file, sep=',')
reviews = raw.copy()

In [41]:
reviews.shape

(5000, 3)

In [42]:
reviews.head(3)

,Unnamed: 0,review_es,sentimiento
0,0,Si está buscando una película de guerra típica...,positivo
1,1,Supongo que algunos directores de películas de...,positivo
2,2,Es difícil contarle más sobre esta película si...,positivo


## 3. Preparación de los datos

In [43]:
# Eliminar la columna de Unnnamed
reviews.drop('Unnamed: 0', axis=1, inplace=True)

In [44]:
# Nulos en el dataset
reviews.isnull().sum()


review_es      0
sentimiento    0
dtype: int64

In [47]:
# Funcion para eliminar caracteres especiales y numeros
def clean_text(text):
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    return text

print(clean_text('Hola, ¿cómo estás?'))

hola   c mo est s 


In [48]:
# Funcion para eliminar caracteres especiales y numeros
def clean_text(text):
    # Pasara a minusculas
    text = text.lower()

    # Tokenizar el texto
    tokens = word_tokenize(text)

    # Regex para eliminar caracteres especiales y numeros
    special_chars = re.compile('[^a-zA-Z\s]')

    # Eliminar caracteres especiales y numeros
    tokens = [special_chars.sub('', word) for word in tokens]

    # Eliminar stopwords
    stop_words = set(stopwords.words('spanish'))

    # Eliminar stopwords
    tokens = [w for w in tokens if not w in stop_words]

    # Reunir los tokens
    text = ' '.join(tokens)

    return text

print(clean_text('Hola, ¿cómo estás?'))

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/PY3/english.pickle[0m

  Searched in:
    - '/Users/juandiegocalixto/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.11/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.11/share/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.11/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************


Para preparar tus datos para utilizar el algoritmo de Naive Bayes en el análisis de sentimientos, necesitas realizar algunas tareas de preprocesamiento de los datos. Los pasos típicos de preprocesamiento incluyen:

- Limpiar el texto: esto implica eliminar cualquier información no relevante para el análisis de sentimientos, como signos de puntuación, números, caracteres especiales y palabras vacías (como "a", "el", "y", etc.). Puedes hacer uso de librerías de procesamiento de texto como NLTK o SpaCy para llevar a cabo esta tarea.
- Tokenizar el texto: esto implica dividir el texto en unidades más pequeñas, como palabras individuales. Puedes hacer uso de las mismas librerías que en el paso anterior.
- Crear una matriz de términos: una vez que tienes tus textos limpios y tokenizados, necesitas crear una matriz que represente la frecuencia de cada término en cada texto. Puedes hacer uso de la clase CountVectorizer de Scikit-Learn para crear esta matriz.
- Dividir los datos en conjuntos de entrenamiento y prueba: debes dividir tus datos en conjuntos de entrenamiento y prueba para poder evaluar el rendimiento del modelo. Puedes hacer uso de la función train_test_split de Scikit-Learn para llevar a cabo esta tarea.
- Aplicar una transformación Tfidf: Puedes aplicar una transformación Tf-idf a la matriz de términos que creaste en el paso 3, esto ayuda a reducir el peso de palabras muy comunes que no aportan información valiosa al modelo. Scikit-Learn tiene una clase TfidfTransformer que puedes utilizar para aplicar esta transformación.
- Una vez que hayas completado estos pasos, estarás listo para entrenar y evaluar tu modelo de Naive Bayes. Puedes seguir los pasos que mencioné anteriormente para utilizar la implementación de Scikit-Learn del algoritmo de Naive Bayes para clasificación de texto.

Claro, para realizar el primer paso de la preparación de los datos, que es la limpieza del texto, puedes seguir los siguientes pasos:

- Eliminar signos de puntuación: utiliza expresiones regulares para eliminar signos de puntuación como comas, puntos, comillas, etc. Estos signos de puntuación no aportan información valiosa para el análisis de sentimientos.
- Eliminar caracteres especiales: elimina cualquier otro carácter especial que no sea un signo de puntuación, como @, #, $, etc.
- Convertir todo el texto a minúsculas: esto ayuda a evitar problemas de inconsistencia en el texto, ya que "Hola" y "hola" se considerarán diferentes palabras si no se convierten a minúsculas.
- Eliminar números: los números no aportan información relevante para el análisis de sentimientos, por lo que es una buena práctica eliminarlos.
- Eliminar palabras vacías: las palabras vacías, como "a", "el", "y", "o", etc., son muy comunes en el lenguaje natural pero no aportan información valiosa para el análisis de sentimientos. Puedes hacer uso de librerías de procesamiento de texto como NLTK o SpaCy para eliminar estas palabras.
- Aplicar stemming: stemming es el proceso de reducir las palabras a su forma base o raíz. Esto ayuda a reducir la cantidad de palabras diferentes que se usan en un texto y simplifica el análisis. NLTK tiene una clase SnowballStemmer que puedes utilizar para llevar a cabo esta tarea.
- Una vez que hayas completado estos pasos, tendrás tus textos limpios y listos para el siguiente paso en la preparación de los datos.

Para eliminar palabras vacías, puedes hacer uso de las librerías NLTK o SpaCy en Python. Aquí te presento un ejemplo de cómo hacerlo con NLTK:

In [49]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('spanish'))

text = "Este es un ejemplo de texto con palabras vacías que deben ser eliminadas"
words = text.split()
filtered_words = [word for word in words if word.lower() not in stop_words]
filtered_text = ' '.join(filtered_words)

print(filtered_text)


[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:992)>


LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/Users/juandiegocalixto/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.11/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.11/share/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.11/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


Para tokenizar el texto utilizando las librerías NLTK o SpaCy en Python, puedes seguir los siguientes ejemplos:

Para tokenizar con NLTK:

In [ ]:
import nltk

text = "Este es un ejemplo de texto que vamos a tokenizar."
tokens = nltk.word_tokenize(text)

print(tokens)


## 4. Implementación algoritmos

## 4.1 Algoritmo Nathalia

## 4.2 Algoritmo Sergio

## 4.3 Algoritmo Calixto

## 5. Análisis de resultados

## 6. Conclusiones